In [21]:
#データの入力とrによる降順ソートを施しておく
import pandas as pd
i = [0,1,2,3]
c = [4,5,12,14]
a = [2,3,5,6]
b = 9
data = {"i":i , "c":c , "a":a}

df = pd.DataFrame(data)
df = df.ix[:, ["i","c","a"]]
df["r"] = df["c"]/df["a"]
df = df.round(2)
df = df.sort_values(by=["r"], ascending=False)
df

,i,c,a,r
2,2,12,5,2.40
3,3,14,6,2.33
0,0,4,2,2.00
1,1,5,3,1.67


In [22]:
i = list(df.i.values)
c = list(df.c.values)
a = list(df.a.values)

In [23]:
#欲張り法によって初めの暫定値を返すメソッド
def greedy(i,c,a,b):
    _sum = 0
    ans = []
    
    for k in range(len(i)):
        if _sum + a[k] <= b:
            ans.append(1)
            _sum = _sum + a[k]
        else:
            ans.append(0)
    return ans

#c,xを与えるとc'xの値を返すメソッド
import numpy as np
def culc(c,x):
    c = np.array(c)
    x = np.array(x)
    return np.dot(c.T,x)

In [24]:
#以上から初期暫定解x,cを得る。
x = greedy(i,c,a,b)
z = culc(c,x)
print ("初期暫定解： x：",x , " z：",z)

初期暫定解： x： [1, 0, 1, 0]  z： 16


In [25]:
#変数の数nと、i,c,a,b、さらに暫定解x,zの組をdataとして保存しておく。
#これ以降で用いるi,c,aは、固定した値を除いた部分問題におけるi,c,aである。
n = 4
data = {"n":n, "i":i[:],"c":c[:],"a":a[:],"b":b,"x":x,"z":z}
data

{'a': [5, 6, 2, 3],
 'b': 9,
 'c': [12, 14, 4, 5],
 'i': [2, 3, 0, 1],
 'n': 4,
 'x': [1, 0, 1, 0],
 'z': 16}

In [26]:
#部分緩和問題を解くメソッド
def gen_solve(i,c,a,j0,j1,data):
    _sum = 0
    b = data['b']
    
    #初期解を一旦全て0に設定しておく
    ans = []
    for k in range(data['n']):
        ans.append(0)

    #j0,j1の処理
    if j0 is not None:
        for k in j0:
            ans[k] = 0
    if j1 is not None:
        for k in j1:
            ans[k] = 1
            b = b - data['a'][data['i'].index(k)]
            _sum = _sum + data['c'][data['i'].index(k)]
    
    #全ての値が固定されているとき
    if len(i) is 0:
        return [ans,_sum]
    
    #固定されていない部分を解く
    for k in range(len(i)):
        
        if b >=  a[k]:
            ans[i[k]] = 1
            b = b - a[k]
            _sum = _sum + c[k]
        
        elif b <= 0:
            ans[i[k]] = 0
        
        else:
            ans[i[k]] = b / a[k]
            b = 0
            _sum = _sum + (c[k]*ans[i[k]])
        
    return [ans,_sum]

In [27]:
#check
gen_solve(i,c,a,[],[],data)

[[0, 0, 1, 0.66666666666666663], 21.333333333333332]

In [28]:
#部分緩和問題の解から、終端せず：0 , 終端:1 , 終端かつ暫定解更新:2 を返すメソッド
def end_judge(ans,_sum,j0,j1,data):

    #解が全て0or1なら終端、暫定解より大なら更新
    flag1 = True
    change = False
    for i in ans:
        if (i is not 0) and (i is not 1):
            flag1 = False
    if flag1 is True:
        if _sum > data['z']:
            change = True
    
    #zよりも_sumが小さければ終端
    flag2 = False
    if data['z'] > _sum:
        flag2 = True
    
    #解が存在しないなら終端、更新は無効
    flag3 = False
    temp = ans[:]
    b = data['b']
    for i in range(len(ans)):
        b = b - (data['a'][data['i'].index(i)] * ans[i])
    if b < 0:
        flag3 = True
        change = False
    
    
    #まとめ
    flag = False
    if (flag1 is True) or (flag2 is True) or (flag3 is True):
        flag = True

    if change is True:
        return 2
    elif flag is True:
        return 1
    else:
        return 0


In [29]:
#check
end_judge([0,0,1,1],26,[],[2,3],data)

1

In [30]:
#check
x = gen_solve(i,c,a,[],[],data)
ans = x[0]
_sum = x[1]
end_judge(ans,_sum,[],[],data)

0

In [31]:
#分岐限定法を解く
def solve(i,c,a,j0,j1,data):
    
    print ("x：", data['x'], "  z：", data['z'], " j0：", j0 , " j1：" , j1)
    
    #解く
    x = gen_solve(i,c,a,j0,j1,data)
    ans = x[0][:]
    _sum = x[1]
    
    print("ans:",ans, " _sum:",_sum)
    
    #終端判定
    y = end_judge(ans,_sum,j0,j1,data)
    
    print("end_judge：",y)
    
    if y is 2:
        data['x'] = ans[:]
        data['z'] = _sum
        return
    elif y is 1:
        return
    
    else:
        
        i = i[:]
        c = c[:]
        a = a[:]
        tmp0 = j0[:]
        tmp1 = j1[:]
        
        tmp0.append(i[0])
        tmp1.append(i[0])
        del i[0]
        del c[0]
        del a[0]
        
        solve(i,c,a,tmp0,j1,data)
        solve(i,c,a,j0,tmp1,data)


In [32]:
solve(i,c,a,[],[],data)

x： [1, 0, 1, 0]   z： 16  j0： []  j1： []
ans: [0, 0, 1, 0.66666666666666663]  _sum: 21.3333333333
end_judge： 0
x： [1, 0, 1, 0]   z： 16  j0： [2]  j1： []
ans: [1, 0.33333333333333331, 0, 1]  _sum: 19.6666666667
end_judge： 0
x： [1, 0, 1, 0]   z： 16  j0： [2, 3]  j1： []
ans: [1, 1, 0, 0]  _sum: 9
end_judge： 1
x： [1, 0, 1, 0]   z： 16  j0： [2]  j1： [3]
ans: [1, 0.33333333333333331, 0, 1]  _sum: 19.6666666667
end_judge： 0
x： [1, 0, 1, 0]   z： 16  j0： [2, 0]  j1： [3]
ans: [0, 1, 0, 1]  _sum: 19
end_judge： 2
x： [0, 1, 0, 1]   z： 19  j0： [2]  j1： [3, 0]
ans: [1, 0.33333333333333331, 0, 1]  _sum: 19.6666666667
end_judge： 0
x： [0, 1, 0, 1]   z： 19  j0： [2, 1]  j1： [3, 0]
ans: [1, 0, 0, 1]  _sum: 18
end_judge： 1
x： [0, 1, 0, 1]   z： 19  j0： [2]  j1： [3, 0, 1]
ans: [1, 1, 0, 1]  _sum: 23
end_judge： 1
x： [0, 1, 0, 1]   z： 19  j0： []  j1： [2]
ans: [0, 0, 1, 0.66666666666666663]  _sum: 21.3333333333
end_judge： 0
x： [0, 1, 0, 1]   z： 19  j0： [3]  j1： [2]
ans: [1, 0.66666666666666663, 1, 0]  _sum: 19.33333

In [20]:
data

{'a': [5, 6, 2, 3],
 'b': 9,
 'c': [12, 14, 4, 5],
 'i': [2, 3, 0, 1],
 'n': 4,
 'x': [0, 1, 0, 1],
 'z': 19}